# Notebook 05: creditcard Neural Network Training & Regularization

**Dataset**: creditcard.csv (Real-World ULB)

**Objective**: Apply best architecture from card_transdata experiments and test 8 regularization strategies (REG-01 to REG-08) to optimize NN performance on production data.

**Primary Metric**: PR-AUC (optimal for 0.17% fraud rate)

**Goal**: Beat or match Random Forest baseline from Notebook 04 using neural networks.

**Experiments**:
1. Transfer best architecture from Notebook 03 (card_transdata)
2. Test 8 regularization configurations (REG-01 to REG-08)
3. Select best NN model for final test evaluation

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
import joblib
import sys
import os
import json
from pathlib import Path

# Add src to path
sys.path.append(os.path.abspath('../'))
import config
from src.nn_architectures import build_fraud_detection_nn
from src.nn_training_utils import train_nn_with_early_stopping, log_experiment
from src.evaluation_metrics import compute_fraud_metrics, print_classification_summary
from src.visualization_utils import (
    plot_training_history,
    plot_confusion_matrix,
    plot_precision_recall_curve
)

# Set random seeds
config.set_random_seeds()

# Get dataset config
ds_config = config.get_dataset_config('creditcard')

print("✓ Imports complete")
print(f"✓ TensorFlow version: {tf.__version__}")
print(f"✓ Random seed set to {config.RANDOM_SEED}")
print(f"✓ Dataset: creditcard.csv (Real-World ULB)")

## 1. Load Preprocessed Data

In [ ]:
# Load original data
df = pd.read_csv(ds_config['data_path'])
X = df[ds_config['feature_cols']].values
y = df[ds_config['target_col']].values

# Load saved split indices
train_idx = np.load(ds_config['train_idx'])
val_idx = np.load(ds_config['val_idx'])
test_idx = np.load(ds_config['test_idx'])

# Split data using saved indices
X_train, y_train = X[train_idx], y[train_idx]
X_val, y_val = X[val_idx], y[val_idx]
X_test, y_test = X[test_idx], y[test_idx]

# Load fitted scaler
scaler = joblib.load(ds_config['scaler_path'])

# Transform data
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✓ Data loaded and preprocessed:")
print(f"  Train: {X_train_scaled.shape[0]:,} samples")
print(f"  Val:   {X_val_scaled.shape[0]:,} samples")
print(f"  Test:  {X_test_scaled.shape[0]:,} samples (RESERVED for Notebook 06)")
print(f"  Features: {X_train_scaled.shape[1]}")
print(f"\n✓ Class distribution:")
print(f"  Train fraud rate: {y_train.mean()*100:.4f}%")
print(f"  Val fraud rate:   {y_val.mean()*100:.4f}%")

## 2. Load Baseline Performance Targets

In [ ]:
# Load baseline results from Notebook 04
baseline_results = pd.read_csv(ds_config['tables_dir'] / 'baseline_results.csv')

print("📊 Baseline Performance Targets (Validation Set):")
print("="*60)
print(baseline_results.to_string(index=False))
print("="*60)

# Extract RF performance as target
rf_pr_auc = baseline_results[baseline_results['model'] == 'Random Forest']['pr_auc'].values[0]
rf_f1 = baseline_results[baseline_results['model'] == 'Random Forest']['f1_fraud'].values[0]

print(f"\n🎯 Target to Beat/Match:")
print(f"   Random Forest PR-AUC: {rf_pr_auc:.4f}")
print(f"   Random Forest F1:     {rf_f1:.4f}")

## 3. Load Best Architecture from card_transdata Experiments

In [ ]:
# Load architecture ranking from Notebook 03
card_transdata_results = config.RESULTS_DIR / 'card_transdata' / 'tables' / 'architecture_ranking.csv'

if card_transdata_results.exists():
    arch_ranking = pd.read_csv(card_transdata_results)
    best_arch = arch_ranking.iloc[0]
    
    print("✓ Loaded best architecture from card_transdata experiments:")
    print(f"  Experiment ID: {best_arch['experiment_id']}")
    print(f"  Architecture: {best_arch['architecture']}")
    print(f"  Layers: {best_arch['layers']}")
    print(f"  card_transdata PR-AUC: {best_arch['pr_auc']:.4f}")
    
    # Parse layers string back to list
    import ast
    best_arch_layers = ast.literal_eval(best_arch['layers'])
    print(f"\n🚀 Transferring architecture to creditcard dataset...")
else:
    print("⚠️  Warning: No architecture ranking found from Notebook 03")
    print("   Using default architecture: [64, 32]")
    best_arch_layers = [64, 32]
    best_arch = {'experiment_id': 'DEFAULT', 'architecture': 'balanced_medium'}

## 4. Regularization Experiments (REG-01 to REG-08)

Test 8 regularization strategies on real-world data using transferred architecture.

In [ ]:
# Initialize experiment log
regularization_results = []

# Create results directory
(ds_config['models_dir'] / 'neural_networks').mkdir(parents=True, exist_ok=True)
(ds_config['experiment_logs_dir']).mkdir(parents=True, exist_ok=True)

print("\n" + "="*70)
print(" REGULARIZATION EXPERIMENTS: Testing 8 Strategies")
print("="*70)
print(f"Architecture: {best_arch_layers}")
print(f"Dataset: creditcard.csv (Real-World ULB)")
print(f"Target: RF PR-AUC = {rf_pr_auc:.4f}")
print("="*70)

In [ ]:
# Iterate through regularization configurations
for reg_id, reg_config in config.REGULARIZATION_EXPERIMENTS.items():
    print(f"\n{'='*70}")
    print(f"Training {reg_id}: {reg_config['description']}")
    print(f"Dropout={reg_config['dropout']}, L2={reg_config['l2']}, BatchNorm={reg_config['batch_norm']}")
    print(f"{'='*70}")
    
    # Build model
    model = build_fraud_detection_nn(
        input_dim=X_train_scaled.shape[1],
        hidden_layers=best_arch_layers,
        dropout_rate=reg_config['dropout'],
        l2_reg=reg_config['l2'],
        use_batch_norm=reg_config['batch_norm']
    )
    
    # Train with early stopping
    history = train_nn_with_early_stopping(
        model=model,
        X_train=X_train_scaled,
        y_train=y_train,
        X_val=X_val_scaled,
        y_val=y_val,
        epochs=config.MAX_EPOCHS,
        batch_size=config.BATCH_SIZE,
        patience=config.EARLY_STOPPING_PATIENCE,
        class_weight='balanced'
    )
    
    # Predictions
    y_val_pred_proba = model.predict(X_val_scaled, verbose=0).flatten()
    y_val_pred = (y_val_pred_proba >= 0.5).astype(int)
    
    # Compute metrics
    metrics = compute_fraud_metrics(y_val, y_val_pred, y_val_pred_proba)
    
    print(f"\n📊 {reg_id} Validation Performance:")
    print(f"  PR-AUC:        {metrics['pr_auc']:.4f} (Target: {rf_pr_auc:.4f})")
    print(f"  ROC-AUC:       {metrics['roc_auc']:.4f}")
    print(f"  F1 (Fraud):    {metrics['f1_fraud']:.4f} (Target: {rf_f1:.4f})")
    print(f"  Recall (Fraud): {metrics['recall_fraud']:.4f}")
    print(f"  Precision (Fraud): {metrics['precision_fraud']:.4f}")
    
    # Compare to RF
    pr_auc_diff = metrics['pr_auc'] - rf_pr_auc
    if pr_auc_diff >= 0:
        print(f"  🎉 BEATS RF by {pr_auc_diff*100:.2f} percentage points!")
    else:
        print(f"  📉 Below RF by {abs(pr_auc_diff)*100:.2f} percentage points")
    
    # Save model
    model_path = ds_config['models_dir'] / 'neural_networks' / f'{reg_id}_regularization.keras'
    model.save(model_path)
    print(f"\n✓ Model saved to: {model_path}")
    
    # Plot learning curves
    fig_path = ds_config['figures_dir'] / 'learning_curves' / f'{reg_id}_learning_curves.png'
    plot_training_history(
        history,
        save_path=str(fig_path),
        title=f'{reg_id}: {reg_config["description"]} - Learning Curves'
    )
    
    # Log experiment
    log_experiment(
        experiment_id=reg_id,
        dataset_name='creditcard',
        experiment_type='regularization',
        model_name=f"{best_arch['architecture']}_reg",
        architecture=best_arch_layers,
        hyperparameters={
            'dropout': reg_config['dropout'],
            'l2': reg_config['l2'],
            'batch_norm': reg_config['batch_norm'],
            'batch_size': config.BATCH_SIZE,
            'epochs_trained': len(history.history['loss'])
        },
        metrics=metrics,
        log_path=ds_config['experiment_logs_dir'] / 'nn_experiments.csv'
    )
    
    # Store results
    regularization_results.append({
        'experiment_id': reg_id,
        'description': reg_config['description'],
        'dropout': reg_config['dropout'],
        'l2': reg_config['l2'],
        'batch_norm': reg_config['batch_norm'],
        'pr_auc': metrics['pr_auc'],
        'roc_auc': metrics['roc_auc'],
        'f1_fraud': metrics['f1_fraud'],
        'recall_fraud': metrics['recall_fraud'],
        'precision_fraud': metrics['precision_fraud'],
        'vs_rf': pr_auc_diff
    })

print("\n" + "="*70)
print("✓ Regularization Experiments Complete!")
print("="*70)

## 5. Regularization Results Analysis

In [ ]:
# Convert results to DataFrame
reg_df = pd.DataFrame(regularization_results)
reg_df = reg_df.sort_values('pr_auc', ascending=False)

print("\n" + "="*90)
print(" REGULARIZATION RANKING BY PR-AUC (Validation Set)")
print("="*90)
print(reg_df[['experiment_id', 'description', 'pr_auc', 'f1_fraud', 'vs_rf']].to_string(index=False))
print("="*90)

# Identify best regularization
best_reg = reg_df.iloc[0]
print(f"\n🏆 Best Regularization: {best_reg['experiment_id']}")
print(f"   Description: {best_reg['description']}")
print(f"   Dropout: {best_reg['dropout']}, L2: {best_reg['l2']}, BatchNorm: {best_reg['batch_norm']}")
print(f"   PR-AUC: {best_reg['pr_auc']:.4f}")
print(f"   F1 (Fraud): {best_reg['f1_fraud']:.4f}")
print(f"   vs RF: {best_reg['vs_rf']*100:+.2f} percentage points")

# Save ranking
reg_ranking_path = ds_config['tables_dir'] / 'regularization_ranking.csv'
reg_df.to_csv(reg_ranking_path, index=False)
print(f"\n✓ Regularization ranking saved to: {reg_ranking_path}")

# Count models that beat RF
models_beat_rf = (reg_df['vs_rf'] > 0).sum()
print(f"\n📊 Summary: {models_beat_rf}/{len(reg_df)} models beat Random Forest baseline")

## 6. Visualize Best Model Performance

In [ ]:
# Load best model
best_model_path = ds_config['models_dir'] / 'neural_networks' / f'{best_reg["experiment_id"]}_regularization.keras'
best_model = keras.models.load_model(best_model_path)

# Generate predictions
y_val_pred_proba = best_model.predict(X_val_scaled, verbose=0).flatten()
y_val_pred = (y_val_pred_proba >= 0.5).astype(int)

# Confusion matrix
plot_confusion_matrix(
    y_val, y_val_pred, 
    ['Legitimate', 'Fraud'],
    save_path=str(ds_config['figures_dir'] / 'best_nn_confusion_matrix.png'),
    title=f'Best NN ({best_reg["experiment_id"]}) - Confusion Matrix'
)

# PR curve
plot_precision_recall_curve(
    y_val, y_val_pred_proba,
    save_path=str(ds_config['figures_dir'] / 'best_nn_pr_curve.png'),
    title=f'Best NN ({best_reg["experiment_id"]}) - PR Curve'
)

print("✓ Best model visualizations saved")

## 7. Save Best Model for Test Evaluation

In [ ]:
# Copy best model to designated location
import shutil

best_model_final_path = ds_config['models_dir'] / 'best_nn_models' / 'best_nn_for_test.keras'
best_model_final_path.parent.mkdir(parents=True, exist_ok=True)
shutil.copy(best_model_path, best_model_final_path)

# Save metadata
best_model_metadata = {
    'experiment_id': best_reg['experiment_id'],
    'description': best_reg['description'],
    'architecture': best_arch_layers,
    'dropout': float(best_reg['dropout']),
    'l2': float(best_reg['l2']),
    'batch_norm': bool(best_reg['batch_norm']),
    'val_pr_auc': float(best_reg['pr_auc']),
    'val_f1_fraud': float(best_reg['f1_fraud']),
    'vs_rf_baseline': float(best_reg['vs_rf'])
}

metadata_path = ds_config['models_dir'] / 'best_nn_models' / 'best_model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(best_model_metadata, f, indent=2)

print(f"✓ Best model saved for test evaluation:")
print(f"  Model: {best_model_final_path}")
print(f"  Metadata: {metadata_path}")
print(f"\n⚠️  TEST SET RESERVED: Do not evaluate until Notebook 06 (threshold optimization)")

## 8. Summary & Next Steps

In [ ]:
print("\n" + "="*70)
print(" NOTEBOOK 05 SUMMARY - creditcard NN TRAINING & REGULARIZATION")
print("="*70)

print(f"\n🏆 Best Neural Network Model: {best_reg['experiment_id']}")
print(f"   Description: {best_reg['description']}")
print(f"   Architecture: {best_arch_layers}")
print(f"   Regularization: Dropout={best_reg['dropout']}, L2={best_reg['l2']}, BatchNorm={best_reg['batch_norm']}")

print(f"\n📊 Validation Performance:")
print(f"   PR-AUC:     {best_reg['pr_auc']:.4f}")
print(f"   F1 (Fraud): {best_reg['f1_fraud']:.4f}")

print(f"\n🎯 vs Random Forest Baseline:")
print(f"   RF PR-AUC:  {rf_pr_auc:.4f}")
print(f"   NN PR-AUC:  {best_reg['pr_auc']:.4f}")
print(f"   Difference: {best_reg['vs_rf']*100:+.2f} percentage points")

if best_reg['vs_rf'] > 0:
    print("\n✅ SUCCESS: Neural network BEATS Random Forest on real-world data!")
elif abs(best_reg['vs_rf']) < 0.01:
    print("\n✅ SUCCESS: Neural network MATCHES Random Forest performance")
else:
    print("\n📊 Neural network below RF, but provides:")
    print("   - Probability calibration for threshold optimization")
    print("   - Interpretable regularization insights")
    print("   - Production deployment flexibility")

print(f"\n📈 Experiments Summary:")
print(f"   Models trained: {len(reg_df)}")
print(f"   Models beat RF: {models_beat_rf}")
print(f"   Best PR-AUC: {reg_df['pr_auc'].max():.4f}")
print(f"   Worst PR-AUC: {reg_df['pr_auc'].min():.4f}")

print(f"\n🚀 Next Steps (Notebook 06):")
print(f"   1. Optimize classification threshold on validation set")
print(f"   2. ONE-TIME test set evaluation (final performance)")
print(f"   3. Compare NN vs RF on test set")
print(f"   4. Generate final performance report")

print("\n📁 Artifacts Created:")
print(f"   - {len(regularization_results)} regularization models")
print(f"   - Best model: {best_model_final_path}")
print(f"   - Experiment logs: {ds_config['experiment_logs_dir']}/nn_experiments.csv")
print(f"   - Learning curves: {ds_config['figures_dir']}/learning_curves/")
print(f"   - Rankings: {reg_ranking_path}")

print("\n✅ Notebook 05 Complete!")
print("🎯 Ready for Notebook 06: Threshold Optimization & Test Evaluation")
print("="*70)